In [1]:
import community as community_louvain
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np

from glob import glob
import operator

import sys


# load the karate club grap

## 대 클러스터에서 질병 클러스터의 중요도 계산하기

In [2]:
#path = glob('./graph/*npy')

In [3]:
path = glob('./*txt')
path = ['./20191230.gkg.csv.txt']

In [ ]:
#theme list 만들기
table = []
f = open('../LOOKUP-GKGTHEMES.txt', 'r')
for line in f.readlines():
    input = line.strip().split('\t')[0]
    table.append(input)

for p in path:
    way = p.split('./')[1].replace('.txt', '')
    sys.stdout = open(f'./output/output{way}.txt','w')
    G1 = nx.Graph()
    file = open(f'{p}', 'r')
    while True:
        line = file.readline()
        if not line:
            break
        line = line.split(',')
        G1.add_edge(str(line[0]), str(line[1]), weight = float(line[2]))
    file.close()

    partition1 = community_louvain.best_partition(G1)

    #partition set을 cluster 갯수대로 만들어놓기
    partition_set = []
    for i in range(len(set(partition1.values()))):
        partition_set.append([])

    for k, v in partition1.items():
        partition_set[v].append(k)

    # theme list를 클러스터별로 담을 리스트 만들기
    partition_set_gkg_list = []
    for i in range(len(set(partition1.values()))):
        partition_set_gkg_list.append([])

    #partition set을 theme list에 맞게 변환 (int 에서 str)
    for i in range(len(partition_set)):
        for k in partition_set[i]:
            partition_set_gkg_list[i].append(table[int(k)])

    print(f'{p} Fisrt Clustering properties: ', G1, '\n')

    for i in range(len(partition_set_gkg_list)):
        print(f"{i} Cluster's number of nodes: ", len(partition_set_gkg_list[i]))
        
        
    for e, i in enumerate (partition_set_gkg_list):
        if 'WB_2167_PANDEMICS' in i:
            int_list = partition_set[e]
            theme_list = i
            np.save('./list', i)
            np.save('./int_list', partition_set[e])  
            print(f'Cluster {e} will be used')

    way = p.split('./')[1].replace('.txt', '')
    
    degree_dict = []
    theme_graph = np.load(f'./graph/theme_graph_{way}.npy')
    
    max_ = np.sum(theme_graph, axis = 0)
    max__ = np.max(max_)
    
    for k in int_list:
        weight = 0
        for i in int_list:
            weight += theme_graph[int(k)][int(i)] / max__
        degree_dict.append(weight) 
        
    alpha_list = [100, 300, 500, 1000]
    
    for alpha in alpha_list:
        sum_ = 0
        i = 0
        for d in degree_dict:
            if i == alpha:
                break
            sum_ += d
            i += 1
        print(f'\n{way} Big {alpha} node degree summation: ', sum_)

        degree_list_dict = {}
        for i in range(len(theme_list)):
            degree_list_dict[theme_list[i]] = degree_dict[i]

        sorted_degree_list= sorted(degree_list_dict.items(), key=operator.itemgetter(1), reverse = True)

        print(sorted_degree_list[:alpha])
        
        
############소분류 클러스터에서 질병 클러스터 중요도 계산하기################
    int_partition_set = []

    for i in int_list:
        int_partition_set.append(int(i))


    G2 = nx.Graph()
    file = open(f'{p}', 'r')
    while True:
        line = file.readline()
        if not line:
            break
        line = line.split(',')
        if int(line[0]) in int_partition_set:
            if int(line[1]) in int_partition_set:
                G2.add_edge(str(line[0]), str(line[1]), weight = float(line[2]))
    file.close()
    

    partition2 = community_louvain.best_partition(G2)

    #partition set을 cluster 갯수대로 만들어놓기
    partition_set = []
    for i in range(len(set(partition2.values()))):
        partition_set.append([])

    for k, v in partition2.items():
        partition_set[v].append(k)

    # theme list를 클러스터별로 담을 리스트 만들기
    partition_set_gkg_list = []
    for i in range(len(set(partition2.values()))):
        partition_set_gkg_list.append([])

    #partition set을 theme list에 맞게 변환 (int 에서 str)
    for i in range(len(partition_set)):
        for k in partition_set[i]:
            partition_set_gkg_list[i].append(table[int(k)])
    print('\n\n################ Second Clustering ################')
    print(f'{p} Second Clustering properties: ', G2, '\n')

    for i in range(len(partition_set_gkg_list)):
        print(f"{i} Cluster's number of nodes: ", len(partition_set_gkg_list[i]))


    for e, i in enumerate (partition_set_gkg_list):
        if 'WB_2167_PANDEMICS' in i:
            int_list = partition_set[e]
            theme_list = i
            np.save('./list', i)
            np.save('./int_list', partition_set[e])  
            print(f'Cluster {e} will be used')

    way = p.split('./')[1].replace('.txt', '')

    degree_dict = []
    theme_graph = np.load(f'./graph/theme_graph_{way}.npy')

    max_ = np.sum(theme_graph, axis = 0)
    max__ = np.max(max_)

    for k in int_list:
        weight = 0
        for i in int_list:
            weight += theme_graph[int(k)][int(i)] / max__
        degree_dict.append(weight) 

    alpha_list = [10, 30, 50, 100]

    for alpha in alpha_list:
        sum_ = 0
        i = 0
        for d in degree_dict:
            if i == alpha:
                break
            sum_ += d
            i += 1
        print(f'\n{way} Small {alpha} node degree summation: ', sum_)

        degree_list_dict = {}
        for i in range(len(theme_list)):
            degree_list_dict[theme_list[i]] = degree_dict[i]

        sorted_degree_list= sorted(degree_list_dict.items(), key=operator.itemgetter(1), reverse = True)

        print(sorted_degree_list[:alpha])
    sys.stdout.close()

## 소분류 클러스터에서 질병 클러스터 중요도 계산하기

In [ ]:
int_partition_set = []

for i in partition_set[e]:
    int_partition_set.append(int(i))

#theme list 만들기
table = []
f = open('../LOOKUP-GKGTHEMES.txt', 'r')
for line in f.readlines():
    input = line.strip().split('\t')[0]
    table.append(input)

sys.stdout = open('./output/output.txt','w')


for p in path:
    
    G1 = nx.Graph()
    file = open(f'{p}', 'r')
    while True:
        line = file.readline()
        if not line:
            break
        line = line.split(',')
        G1.add_edge(str(line[0]), str(line[1]), weight = float(line[2]))
    file.close()

    partition1 = community_louvain.best_partition(G1)

    #partition set을 cluster 갯수대로 만들어놓기
    partition_set = []
    for i in range(len(set(partition1.values()))):
        partition_set.append([])

    for k, v in partition1.items():
        partition_set[v].append(k)

    # theme list를 클러스터별로 담을 리스트 만들기
    partition_set_gkg_list = []
    for i in range(len(set(partition1.values()))):
        partition_set_gkg_list.append([])

    #partition set을 theme list에 맞게 변환 (int 에서 str)
    for i in range(len(partition_set)):
        for k in partition_set[i]:
            partition_set_gkg_list[i].append(table[int(k)])

    print(f'{p} Clustering properties: ', G1, '\n')

    for i in range(len(partition_set_gkg_list)):
        print(f"{i} Cluster's number of nodes: ", len(partition_set_gkg_list[i]))
        
        
    for e, i in enumerate (partition_set_gkg_list):
        if 'WB_2167_PANDEMICS' in i:
            int_list = partition_set[e]
            theme_list = i
            np.save('./list', i)
            np.save('./int_list', partition_set[e])  
            print(f'Cluster {e} will be used')

    way = p.split('./')[1].replace('.txt', '')
    
    degree_dict = []
    theme_graph = np.load(f'./graph/theme_graph_{way}.npy')
    
    max_ = np.sum(theme_graph, axis = 0)
    max__ = np.max(max_)
    
    for k in int_list:
        weight = 0
        for i in int_list:
            weight += theme_graph[int(k)][int(i)] / max__
        degree_dict.append(weight) 
        
    sum_ = 0
    i = 0
    for d in degree_dict:
        if i == 100:
            break
        sum_ += d
        i += 1
    print(f'\n{way} upper 100 node degree summation: ', sum_)
    
    degree_list_dict = {}
    for i in range(len(theme_list)):
        degree_list_dict[theme_list[i]] = degree_dict[i]
        
    sorted_degree_list= sorted(degree_list_dict.items(), key=operator.itemgetter(1), reverse = True)

    print(sorted_degree_list[:100])

In [154]:
int_partition_set = []

for i in partition_set[3]:
    int_partition_set.append(int(i))

In [155]:
G1_disease = nx.Graph()
file = open('./corona_before_graph.txt', 'r')

while True:
    line = file.readline()
    if not line:
        break
    line = line.split(',')
    if int(line[0]) in int_partition_set:
        if int(line[1]) in int_partition_set:
            G1_disease.add_edge(str(line[0]), str(line[1]), weight = float(line[2]))
file.close()

In [156]:
partition_G1_disease = community_louvain.best_partition(G1_disease)

In [157]:
partition_set = []
for i in range(len(set(partition_G1_disease.values()))):
    partition_set.append([])

for k, v in partition_G1_disease.items():
    partition_set[v].append(k)
    
#theme list 만들기
table = []
f = open('./LOOKUP-GKGTHEMES.txt', 'r')
for line in f.readlines():
    input = line.strip().split('\t')[0]
    table.append(input)
    
# theme list를 클러스터별로 담을 리스트 만들기

partition_set_gkg_list = []
for i in range(len(set(partition_G1_disease.values()))):
    partition_set_gkg_list.append([])
    
#partition set을 theme list에 맞게 변환 (int 에서 str)
for i in range(len(partition_set)):
    for k in partition_set[i]:
        partition_set_gkg_list[i].append(table[int(k)])
        
print('Clustering properties: ', G1_disease, '\n')

for i in range(len(partition_set_gkg_list)):
    print(f"{i} Cluster's number of nodes: ", len(partition_set_gkg_list[i]))

Clustering properties:  Graph with 2634 nodes and 172419 edges 

0 Cluster's number of nodes:  565
1 Cluster's number of nodes:  338
2 Cluster's number of nodes:  720
3 Cluster's number of nodes:  893
4 Cluster's number of nodes:  118


In [164]:
partition_set_gkg_list[2]

['WB_621_HEALTH_NUTRITION_AND_POPULATION',
 'WB_137_WATER',
 'WB_635_PUBLIC_HEALTH',
 'TAX_FOODSTAPLES',
 'WB_1458_HEALTH_PROMOTION_AND_DISEASE_PREVENTION',
 'WB_1462_WATER_SANITATION_AND_HYGIENE',
 'CRISISLEX_C06_WATER_SANITATION',
 'WB_2165_HEALTH_EMERGENCIES',
 'CRISISLEX_O02_RESPONSEAGENCIESATCRISIS',
 'WB_644_NUTRITION',
 'DISABILITY',
 'WB_1464_HEALTH_OF_THE_DISABLED',
 'WATER_SECURITY',
 'TAX_AIDGROUPS',
 'WB_1441_SUPPLEMENTS',
 'WB_1324_EMERGENCY_MEDICAL_SERVICES',
 'TAX_FOODSTAPLES_MEAT',
 'WB_2167_PANDEMICS',
 'WB_2166_HEALTH_EMERGENCY_PREPAREDNESS_AND_DISASTER_RESPONSE',
 'TAX_WORLDINSECTS',
 'CRISISLEX_T06_SUPPLIES',
 'ECON_DEVELOPMENTORGS',
 'WB_1199_WATER_SUPPLY_AND_SANITATION',
 'TAX_FOODSTAPLES_RICE',
 'TAX_FNCACT_COOK',
 'FOOD_SECURITY',
 'TAX_FOODSTAPLES_MILK',
 'WB_639_REPRODUCTIVE_MATERNAL_AND_CHILD_HEALTH',
 'WB_139_SANITATION_AND_WASTEWATER',
 'TAX_DISEASE_OUTBREAK',
 'DISPLACED',
 'TAX_DISEASE_FEVER',
 'TAX_DISEASE_FLU',
 'CRISISLEX_T07_SERVICESNEEDEDOFFERED',
 '

In [165]:
int_partition_set = []

for i in partition_set[2]:
    int_partition_set.append(int(i))

In [166]:
G2_disease_disease = nx.Graph()
file = open('./corona_before_graph.txt', 'r')

while True:
    line = file.readline()
    if not line:
        break
    line = line.split(',')
    if int(line[0]) in int_partition_set:
        if int(line[1]) in int_partition_set:
            G2_disease_disease.add_edge(str(line[0]), str(line[1]), weight = float(line[2]))
file.close()

In [167]:
partition_G2_disease = community_louvain.best_partition(G2_disease_disease)

In [168]:
partition_set = []
for i in range(len(set(partition_G2_disease.values()))):
    partition_set.append([])

for k, v in partition_G2_disease.items():
    partition_set[v].append(k)
    
#theme list 만들기
table = []
f = open('./LOOKUP-GKGTHEMES.txt', 'r')
for line in f.readlines():
    input = line.strip().split('\t')[0]
    table.append(input)
    
# theme list를 클러스터별로 담을 리스트 만들기

partition_set_gkg_list = []
for i in range(len(set(partition_G2_disease.values()))):
    partition_set_gkg_list.append([])
    
#partition set을 theme list에 맞게 변환 (int 에서 str)
for i in range(len(partition_set)):
    for k in partition_set[i]:
        partition_set_gkg_list[i].append(table[int(k)])
        
print('Clustering properties: ', G2_disease_disease, '\n')

for i in range(len(partition_set_gkg_list)):
    print(f"{i} Cluster's number of nodes: ", len(partition_set_gkg_list[i]))

Clustering properties:  Graph with 720 nodes and 16612 edges 

0 Cluster's number of nodes:  81
1 Cluster's number of nodes:  148
2 Cluster's number of nodes:  125
3 Cluster's number of nodes:  279
4 Cluster's number of nodes:  87


In [175]:
partition_set_gkg_list[3]

['WB_2165_HEALTH_EMERGENCIES',
 'TAX_AIDGROUPS',
 'WB_2167_PANDEMICS',
 'WB_2166_HEALTH_EMERGENCY_PREPAREDNESS_AND_DISASTER_RESPONSE',
 'ECON_DEVELOPMENTORGS',
 'WB_639_REPRODUCTIVE_MATERNAL_AND_CHILD_HEALTH',
 'TAX_DISEASE_OUTBREAK',
 'TAX_DISEASE_FEVER',
 'TAX_DISEASE_FLU',
 'HEALTH_VACCINATION',
 'WB_1415_COMMUNICABLE_DISEASE',
 'WB_642_CHILD_HEALTH',
 'WB_1459_IMMUNIZATIONS',
 'STATE_OF_EMERGENCY',
 'TAX_DISEASE_EPIDEMIC',
 'TAX_DISEASE_INFECTIOUS',
 'ECON_DEVELOPMENTORGS_WORLD_HEALTH_ORGANIZATION',
 'TAX_AIDGROUPS_WORLD_HEALTH_ORGANIZATION',
 'SOC_QUARANTINE',
 'TAX_DISEASE_HEADACHE',
 'TAX_DISEASE_PARALYSIS',
 'TAX_DISEASE_POISONED',
 'TAX_DISEASE_DIARRHEA',
 'TAX_DISEASE_CONTAGIOUS',
 'SICKENED',
 'TAX_AIDGROUPS_UNICEF',
 'TAX_DISEASE_EBOLA',
 'WB_2663_EBOLA',
 'TAX_DISEASE_INFLUENZA',
 'WB_1421_INFLUENZA',
 'TAX_DISEASE_MEASLES',
 'TAX_DISEASE_VOMITING',
 'TAX_DISEASE_MALARIA',
 'WB_1420_MALARIA',
 'TAX_DISEASE_RABIES',
 'TAX_DISEASE_PATHOGENS',
 'TAX_DISEASE_CHOLERA',
 'TAX_FN

In [174]:
partition_set[3]

['274',
 '391',
 '478',
 '479',
 '509',
 '625',
 '657',
 '688',
 '694',
 '731',
 '747',
 '824',
 '856',
 '937',
 '1006',
 '1039',
 '1162',
 '1176',
 '1347',
 '1440',
 '1475',
 '1510',
 '1571',
 '1607',
 '1658',
 '1661',
 '1703',
 '1734',
 '1754',
 '1763',
 '1815',
 '1854',
 '1959',
 '1972',
 '1985',
 '2038',
 '2067',
 '2151',
 '2222',
 '2278',
 '2305',
 '2325',
 '2375',
 '2723',
 '2768',
 '2771',
 '2784',
 '2812',
 '2846',
 '2905',
 '2915',
 '2937',
 '2988',
 '3094',
 '3098',
 '3107',
 '3117',
 '3167',
 '3190',
 '3200',
 '3208',
 '3350',
 '3377',
 '3484',
 '3492',
 '3514',
 '3544',
 '3583',
 '3640',
 '3721',
 '3748',
 '3755',
 '3778',
 '3795',
 '3814',
 '3816',
 '3823',
 '3845',
 '3886',
 '3965',
 '4003',
 '4047',
 '4048',
 '4067',
 '4173',
 '4186',
 '4204',
 '4216',
 '4243',
 '4333',
 '4359',
 '4382',
 '4397',
 '4447',
 '4456',
 '4459',
 '4472',
 '4474',
 '4549',
 '4598',
 '4702',
 '4708',
 '4791',
 '4800',
 '4837',
 '4868',
 '4887',
 '4908',
 '4909',
 '4953',
 '5013',
 '5033',
 '5076

## Corona(after)

In [8]:
G2 = nx.Graph()
file = open('../corona_after_graph.txt', 'r')
while True:
    line = file.readline()
    if not line:
        break
    line = line.split(',')
    G2.add_edge(str(line[0]), str(line[1]), weight = float(line[2]))
file.close()

In [9]:
# compute the best partition
partition2 = community_louvain.best_partition(G2)

In [10]:
#partition set을 cluster 갯수대로 만들어놓기
partition_set = []
for i in range(len(set(partition2.values()))):
    partition_set.append([])

for k, v in partition2.items():
    partition_set[v].append(k)

In [12]:
#theme list 만들기
table = []
f = open('../LOOKUP-GKGTHEMES.txt', 'r')
for line in f.readlines():
    input = line.strip().split('\t')[0]
    table.append(input)

In [13]:
# theme list를 클러스터별로 담을 리스트 만들기

partition_set_gkg_list = []
for i in range(len(set(partition2.values()))):
    partition_set_gkg_list.append([])
    
#partition set을 theme list에 맞게 변환 (int 에서 str)
for i in range(len(partition_set)):
    for k in partition_set[i]:
        partition_set_gkg_list[i].append(table[int(k)])

In [14]:
print('Clustering properties: ', G2, '\n')

for i in range(len(partition_set_gkg_list)):
    print(f"{i} Cluster's number of nodes: ", len(partition_set_gkg_list[i]))

Clustering properties:  Graph with 9567 nodes and 2388545 edges 

0 Cluster's number of nodes:  813
1 Cluster's number of nodes:  160
2 Cluster's number of nodes:  2498
3 Cluster's number of nodes:  2174
4 Cluster's number of nodes:  2284
5 Cluster's number of nodes:  1638


In [15]:
for i in partition_set_gkg_list[3]:
    print(i)

SOC_POINTSOFINTEREST
EDUCATION
GENERAL_HEALTH
MEDICAL
WB_621_HEALTH_NUTRITION_AND_POPULATION
CRISISLEX_C03_WELLBEING_HEALTH
TAX_DISEASE
CRISISLEX_T11_UPDATESSYMPATHY
TAX_FNCACT_CHILDREN
SOC_POINTSOFINTEREST_SCHOOL
WB_1406_DISEASES
SCIENCE
SOC_POINTSOFINTEREST_HOSPITAL
TAX_FNCACT_WOMEN
SOC_POINTSOFINTEREST_UNIVERSITY
WB_1427_NON_COMMUNICABLE_DISEASE_AND_INJURY
WB_1331_HEALTH_TECHNOLOGIES
WB_635_PUBLIC_HEALTH
WB_1350_PHARMACEUTICALS
TAX_FNCACT_CHILD
WB_697_SOCIAL_PROTECTION_AND_LABOR
TAX_FNCACT_STUDENTS
WB_2453_ORGANIZED_CRIME
WB_1458_HEALTH_PROMOTION_AND_DISEASE_PREVENTION
WB_2456_DRUGS_AND_NARCOTICS
TAX_FNCACT_AUTHOR
WB_1466_SOCIAL_ASSISTANCE
SOC_POINTSOFINTEREST_COLLEGE
UNGP_HEALTHCARE
TAX_FNCACT_STUDENT
SOC_POINTSOFINTEREST_SCHOOLS
TAX_FNCACT_DOCTOR
TAX_FNCACT_PROFESSOR
TAX_FNCACT_TEACHER
WB_2165_HEALTH_EMERGENCIES
WB_2026_PREVENTION
TAX_FNCACT_ASSISTANT
WB_1609_FOOD_AND_IN_KIND_TRANSFERS
EPU_CATS_HEALTHCARE
WB_644_NUTRITION
WB_1497_EDUCATION_MANAGEMENT_AND_ADMINISTRATION
DISABILITY


In [190]:
partition_set[3]

['6',
 '14',
 '17',
 '19',
 '20',
 '34',
 '35',
 '43',
 '57',
 '59',
 '64',
 '71',
 '85',
 '86',
 '87',
 '88',
 '90',
 '91',
 '92',
 '97',
 '101',
 '121',
 '148',
 '150',
 '153',
 '156',
 '210',
 '211',
 '219',
 '226',
 '231',
 '244',
 '252',
 '262',
 '274',
 '276',
 '280',
 '282',
 '288',
 '294',
 '299',
 '307',
 '310',
 '311',
 '320',
 '324',
 '327',
 '328',
 '334',
 '337',
 '340',
 '346',
 '350',
 '364',
 '391',
 '404',
 '413',
 '422',
 '434',
 '437',
 '467',
 '478',
 '479',
 '486',
 '488',
 '489',
 '492',
 '498',
 '512',
 '534',
 '540',
 '541',
 '544',
 '547',
 '548',
 '549',
 '551',
 '555',
 '562',
 '567',
 '568',
 '571',
 '576',
 '603',
 '604',
 '614',
 '625',
 '628',
 '632',
 '635',
 '636',
 '640',
 '648',
 '650',
 '653',
 '657',
 '664',
 '666',
 '668',
 '677',
 '688',
 '694',
 '698',
 '700',
 '710',
 '721',
 '724',
 '731',
 '738',
 '743',
 '747',
 '748',
 '750',
 '758',
 '762',
 '764',
 '767',
 '776',
 '781',
 '783',
 '788',
 '790',
 '796',
 '805',
 '810',
 '811',
 '813',
 '814

In [16]:
int_partition_set = []

for i in partition_set[3]:
    int_partition_set.append(int(i))

## 질병 클러스터를 다시 클러스터링 하기

In [18]:
G2_disease = nx.Graph()
file = open('../corona_after_graph.txt', 'r')

while True:
    line = file.readline()
    if not line:
        break
    line = line.split(',')
    if int(line[0]) in int_partition_set:
        if int(line[1]) in int_partition_set:
            G2_disease.add_edge(str(line[0]), str(line[1]), weight = float(line[2]))
file.close()


In [19]:
partition_G2_disease = community_louvain.best_partition(G2_disease)

In [21]:
partition_set = []
for i in range(len(set(partition_G2_disease.values()))):
    partition_set.append([])

for k, v in partition_G2_disease.items():
    partition_set[v].append(k)
    
#theme list 만들기
table = []
f = open('../LOOKUP-GKGTHEMES.txt', 'r')
for line in f.readlines():
    input = line.strip().split('\t')[0]
    table.append(input)
    
# theme list를 클러스터별로 담을 리스트 만들기

partition_set_gkg_list = []
for i in range(len(set(partition_G2_disease.values()))):
    partition_set_gkg_list.append([])
    
#partition set을 theme list에 맞게 변환 (int 에서 str)
for i in range(len(partition_set)):
    for k in partition_set[i]:
        partition_set_gkg_list[i].append(table[int(k)])
        
print('Clustering properties: ', G2_disease, '\n')

for i in range(len(partition_set_gkg_list)):
    print(f"{i} Cluster's number of nodes: ", len(partition_set_gkg_list[i]))

Clustering properties:  Graph with 2174 nodes and 128346 edges 

0 Cluster's number of nodes:  530
1 Cluster's number of nodes:  78
2 Cluster's number of nodes:  675
3 Cluster's number of nodes:  120
4 Cluster's number of nodes:  587
5 Cluster's number of nodes:  184


In [27]:
for i in partition_set_gkg_list[5]:
    print(i)

WB_697_SOCIAL_PROTECTION_AND_LABOR
WB_1466_SOCIAL_ASSISTANCE
WB_1609_FOOD_AND_IN_KIND_TRANSFERS
WB_1618_FOOD_DISTRIBUTION
WB_1620_ELDERLY
WB_1608_CASH_TRANSFERS
WB_1603_FEE_WAIVERS
WB_1448_DEMOGRAPHIC_CHANGE
WB_1614_NUTRITIONAL_PROGRAMS
WB_643_AGING_POPULATION
TAX_WORLDMAMMALS_MOUSE
WB_1615_THERAPEUTIC
WB_756_VULNERABLE_GROUPS
WB_758_ORPHANS
WB_1447_GERIATRICS
WB_1625_SUPPORT_SERVICES
WB_2666_LIFE_EXPECTANCY
TAX_FNCACT_NOTARY
WB_1607_HEALTH_BENEFITS
SOC_AGINGPOPULATION
UNGP_SUPPORT_FOR_PEOPLE_WHO_CANT_WORK
NATURAL_DISASTER_TREMORS
TAX_WORLDFISH_CARP
TAX_DISEASE_ATROPHY
TAX_FNCACT_AVIATOR
WB_1613_FOOD_STAMPS
TAX_DISEASE_PARKINSON_DISEASE
WB_2669_PREMATURE_DEATH
TAX_DISEASE_RHEUMATISM
TAX_WORLDLANGUAGES_LANGE
WB_1623_PRICE_SUBSIDIES
WB_1624_SOCIAL_PENSIONS
TAX_WORLDLANGUAGES_SENECA
TAX_DISEASE_PSORIASIS
TAX_DISEASE_COLITIS
TAX_DISEASE_NEUROPATHY
TAX_FNCACT_DIETICIAN
TAX_DISEASE_LUPUS
TAX_FNCACT_FIDDLER
TAX_DISEASE_MUSCULAR_DYSTROPHY
TAX_DISEASE_RARE_DISEASES
WB_1636_DISABILITY_ALLOWANCE


In [198]:
int_partition_set = []

for i in partition_set[4]:
    int_partition_set.append(int(i))

In [199]:
G2_disease_disease = nx.Graph()
file = open('./corona_after_graph.txt', 'r')

while True:
    line = file.readline()
    if not line:
        break
    line = line.split(',')
    if int(line[0]) in int_partition_set:
        if int(line[1]) in int_partition_set:
            G2_disease_disease.add_edge(str(line[0]), str(line[1]), weight = float(line[2]))
file.close()

In [200]:
partition_G2_disease = community_louvain.best_partition(G2_disease_disease)

In [201]:
partition_set = []
for i in range(len(set(partition_G2_disease.values()))):
    partition_set.append([])

for k, v in partition_G2_disease.items():
    partition_set[v].append(k)
    
#theme list 만들기
table = []
f = open('./LOOKUP-GKGTHEMES.txt', 'r')
for line in f.readlines():
    input = line.strip().split('\t')[0]
    table.append(input)
    
# theme list를 클러스터별로 담을 리스트 만들기

partition_set_gkg_list = []
for i in range(len(set(partition_G2_disease.values()))):
    partition_set_gkg_list.append([])
    
#partition set을 theme list에 맞게 변환 (int 에서 str)
for i in range(len(partition_set)):
    for k in partition_set[i]:
        partition_set_gkg_list[i].append(table[int(k)])
        
print('Clustering properties: ', G2_disease_disease, '\n')

for i in range(len(partition_set_gkg_list)):
    print(f"{i} Cluster's number of nodes: ", len(partition_set_gkg_list[i]))

Clustering properties:  Graph with 802 nodes and 25008 edges 

0 Cluster's number of nodes:  200
1 Cluster's number of nodes:  126
2 Cluster's number of nodes:  259
3 Cluster's number of nodes:  165
4 Cluster's number of nodes:  52


In [205]:
partition_set_gkg_list[2]

['UNGP_HEALTHCARE',
 'WB_2165_HEALTH_EMERGENCIES',
 'WB_2026_PREVENTION',
 'TAX_DISEASE_DISEASE',
 'WB_2167_PANDEMICS',
 'WB_2166_HEALTH_EMERGENCY_PREPAREDNESS_AND_DISASTER_RESPONSE',
 'TAX_DISEASE_DISEASES',
 'WB_639_REPRODUCTIVE_MATERNAL_AND_CHILD_HEALTH',
 'TAX_DISEASE_OUTBREAK',
 'TAX_DISEASE_FEVER',
 'TAX_DISEASE_FLU',
 'HEALTH_VACCINATION',
 'WB_1415_COMMUNICABLE_DISEASE',
 'WB_642_CHILD_HEALTH',
 'WB_1459_IMMUNIZATIONS',
 'TAX_DISEASE_COUGH',
 'TAX_DISEASE_EPIDEMIC',
 'TAX_DISEASE_INFECTIOUS',
 'HEALTH_SEXTRANSDISEASE',
 'TAX_DISEASE_HIV',
 'SOC_QUARANTINE',
 'TAX_DISEASE_PNEUMONIA',
 'TAX_DISEASE_PLAGUE',
 'TAX_DISEASE_CONTAGIOUS',
 'TAX_DISEASE_HEPATITIS',
 'SICKENED',
 'TAX_DISEASE_EBOLA',
 'WB_2663_EBOLA',
 'TAX_DISEASE_INFLUENZA',
 'WB_1421_INFLUENZA',
 'TAX_DISEASE_MEASLES',
 'SOC_POINTSOFINTEREST_HEALTH_CENTERS',
 'TAX_DISEASE_MALARIA',
 'WB_1420_MALARIA',
 'TAX_DISEASE_RABIES',
 'TAX_DISEASE_CHOLERA',
 'TAX_DISEASE_TUBERCULOSIS',
 'WB_1419_TUBERCULOSIS',
 'TAX_FNCACT_VET

In [207]:
partition_set[2]

['219',
 '274',
 '276',
 '327',
 '478',
 '479',
 '547',
 '625',
 '657',
 '688',
 '694',
 '731',
 '747',
 '824',
 '856',
 '930',
 '1006',
 '1039',
 '1140',
 '1201',
 '1347',
 '1416',
 '1434',
 '1607',
 '1618',
 '1658',
 '1703',
 '1734',
 '1754',
 '1763',
 '1815',
 '1869',
 '1959',
 '1972',
 '1985',
 '2067',
 '2144',
 '2155',
 '2157',
 '2180',
 '2278',
 '2305',
 '2325',
 '2375',
 '2450',
 '2466',
 '2511',
 '2638',
 '2669',
 '2700',
 '2768',
 '2771',
 '2784',
 '2838',
 '2847',
 '2937',
 '2942',
 '2988',
 '2999',
 '3008',
 '3077',
 '3107',
 '3117',
 '3164',
 '3190',
 '3303',
 '3327',
 '3334',
 '3350',
 '3377',
 '3400',
 '3472',
 '3484',
 '3514',
 '3544',
 '3560',
 '3586',
 '3721',
 '3748',
 '3795',
 '3814',
 '3816',
 '3823',
 '3829',
 '4018',
 '4032',
 '4047',
 '4067',
 '4068',
 '4074',
 '4102',
 '4173',
 '4186',
 '4188',
 '4235',
 '4241',
 '4299',
 '4368',
 '4397',
 '4406',
 '4472',
 '4528',
 '4549',
 '4587',
 '4588',
 '4598',
 '4613',
 '4708',
 '4773',
 '4851',
 '4887',
 '4907',
 '4909',

## zika(before)

In [6]:
G3 = nx.Graph()
file = open('../zika_before_graph.txt', 'r')
while True:
    line = file.readline()
    if not line:
        break
    line = line.split(',')
    G3.add_edge(str(line[0]), str(line[1]), weight = float(line[2]))
file.close()

# compute the best partition
partition3 = community_louvain.best_partition(G3)


In [209]:
#partition set을 cluster 갯수대로 만들어놓기
partition_set = []
for i in range(len(set(partition3.values()))):
    partition_set.append([])

for k, v in partition3.items():
    partition_set[v].append(k)
    
#theme list 만들기
table = []
f = open('./LOOKUP-GKGTHEMES.txt', 'r')
for line in f.readlines():
    input = line.strip().split('\t')[0]
    table.append(input)
    
# theme list를 클러스터별로 담을 리스트 만들기

partition_set_gkg_list = []
for i in range(len(set(partition3.values()))):
    partition_set_gkg_list.append([])
    
#partition set을 theme list에 맞게 변환 (int 에서 str)
for i in range(len(partition_set)):
    for k in partition_set[i]:
        partition_set_gkg_list[i].append(table[int(k)])
        
print('Clustering properties: ', G3, '\n')

for i in range(len(partition_set_gkg_list)):
    print(f"{i} Cluster's number of nodes: ", len(partition_set_gkg_list[i]))

Clustering properties:  Graph with 4693 nodes and 658721 edges 

0 Cluster's number of nodes:  1236
1 Cluster's number of nodes:  441
2 Cluster's number of nodes:  1121
3 Cluster's number of nodes:  1210
4 Cluster's number of nodes:  214
5 Cluster's number of nodes:  471


In [216]:
for i in partition_set_gkg_list[3]:
    print(i)

SOC_POINTSOFINTEREST
EDUCATION
GENERAL_HEALTH
MEDICAL
WB_621_HEALTH_NUTRITION_AND_POPULATION
CRISISLEX_C03_WELLBEING_HEALTH
TAX_DISEASE
SOC_POINTSOFINTEREST_SCHOOL
WB_1406_DISEASES
WB_137_WATER
SCIENCE
NATURAL_DISASTER
SOC_POINTSOFINTEREST_HOSPITAL
TAX_FNCACT_WOMEN
SOC_POINTSOFINTEREST_UNIVERSITY
WB_1427_NON_COMMUNICABLE_DISEASE_AND_INJURY
WB_435_AGRICULTURE_AND_FOOD_SECURITY
WB_1331_HEALTH_TECHNOLOGIES
WB_635_PUBLIC_HEALTH
WB_1350_PHARMACEUTICALS
TAX_FNCACT_CHILD
TAX_FNCACT_WOMAN
TAX_FNCACT_STUDENTS
WB_2453_ORGANIZED_CRIME
WB_1458_HEALTH_PROMOTION_AND_DISEASE_PREVENTION
WB_2456_DRUGS_AND_NARCOTICS
TAX_FNCACT_AUTHOR
WB_1428_INJURY
WB_566_ENVIRONMENT_AND_NATURAL_RESOURCES
WB_1967_AGRICULTURAL_RISK_AND_SECURITY
CRISISLEX_O01_WEATHER
WB_1466_SOCIAL_ASSISTANCE
SOC_POINTSOFINTEREST_COLLEGE
TAX_FNCACT_DRIVER
WB_1973_FINANCIAL_RISK_REDUCTION
WB_337_INSURANCE
TAX_FNCACT_STUDENT
WB_1979_NATURAL_RESOURCE_MANAGEMENT
SOC_POINTSOFINTEREST_SCHOOLS
CRISISLEX_T04_INFRASTRUCTURE
WB_590_ECOSYSTEMS
TAX_F

## zika(after)

In [218]:
G4 = nx.Graph()
file = open('./zika_after_graph.txt', 'r')
while True:
    line = file.readline()
    if not line:
        break
    line = line.split(',')
    G4.add_edge(str(line[0]), str(line[1]), weight = float(line[2]))
file.close()

# compute the best partition
partition4 = community_louvain.best_partition(G4)


In [219]:
#partition set을 cluster 갯수대로 만들어놓기
partition_set = []
for i in range(len(set(partition4.values()))):
    partition_set.append([])

for k, v in partition4.items():
    partition_set[v].append(k)
    
#theme list 만들기
table = []
f = open('./LOOKUP-GKGTHEMES.txt', 'r')
for line in f.readlines():
    input = line.strip().split('\t')[0]
    table.append(input)
    
# theme list를 클러스터별로 담을 리스트 만들기

partition_set_gkg_list = []
for i in range(len(set(partition4.values()))):
    partition_set_gkg_list.append([])
    
#partition set을 theme list에 맞게 변환 (int 에서 str)
for i in range(len(partition_set)):
    for k in partition_set[i]:
        partition_set_gkg_list[i].append(table[int(k)])
        
print('Clustering properties: ', G4, '\n')

for i in range(len(partition_set_gkg_list)):
    print(f"{i} Cluster's number of nodes: ", len(partition_set_gkg_list[i]))

Clustering properties:  Graph with 10043 nodes and 2799008 edges 

0 Cluster's number of nodes:  4749
1 Cluster's number of nodes:  2626
2 Cluster's number of nodes:  1842
3 Cluster's number of nodes:  826


In [222]:
for i in partition_set_gkg_list[2]:
    print(i)

SOC_POINTSOFINTEREST
EDUCATION
GENERAL_HEALTH
MEDICAL
WB_621_HEALTH_NUTRITION_AND_POPULATION
TAX_WORLDMAMMALS
CRISISLEX_C03_WELLBEING_HEALTH
TAX_DISEASE
TAX_FNCACT_CHILDREN
SOC_POINTSOFINTEREST_SCHOOL
WB_1406_DISEASES
TAX_WORLDMAMMALS_HUMAN
SOC_POINTSOFINTEREST_HOSPITAL
TAX_FNCACT_WOMEN
SOC_POINTSOFINTEREST_UNIVERSITY
WB_1427_NON_COMMUNICABLE_DISEASE_AND_INJURY
WB_1331_HEALTH_TECHNOLOGIES
WB_635_PUBLIC_HEALTH
WB_1350_PHARMACEUTICALS
TAX_FNCACT_CHILD
WB_697_SOCIAL_PROTECTION_AND_LABOR
TAX_FNCACT_STUDENTS
WB_2453_ORGANIZED_CRIME
WB_1458_HEALTH_PROMOTION_AND_DISEASE_PREVENTION
WB_2456_DRUGS_AND_NARCOTICS
WB_1428_INJURY
WB_1967_AGRICULTURAL_RISK_AND_SECURITY
WB_1466_SOCIAL_ASSISTANCE
SOC_POINTSOFINTEREST_COLLEGE
TAX_FNCACT_KING
WB_1973_FINANCIAL_RISK_REDUCTION
WB_337_INSURANCE
TAX_FNCACT_STUDENT
TAX_ETHNICITY_ENGLISH
TAX_WORLDLANGUAGES_ENGLISH
SOC_POINTSOFINTEREST_SCHOOLS
TAX_FNCACT_DOCTOR
WB_1462_WATER_SANITATION_AND_HYGIENE
TAX_FNCACT_PROFESSOR
TAX_FNCACT_TEACHER
WB_2165_HEALTH_EMERGENCI

## ebola(before)

In [187]:
G5 = nx.Graph()
file = open('./ebola_before_graph.txt', 'r')
while True:
    line = file.readline()
    if not line:
        break
    line = line.split(',')
    G5.add_edge(str(line[0]), str(line[1]), weight = float(line[2]))
file.close()

# compute the best partition
partition5 = community_louvain.best_partition(G5)



Clustering properties:  Graph with 3131 nodes and 319127 edges 

0 Cluster's number of nodes:  1588
1 Cluster's number of nodes:  708
2 Cluster's number of nodes:  371
3 Cluster's number of nodes:  464


In [223]:
#partition set을 cluster 갯수대로 만들어놓기
partition_set = []
for i in range(len(set(partition5.values()))):
    partition_set.append([])

for k, v in partition5.items():
    partition_set[v].append(k)
    
#theme list 만들기
table = []
f = open('./LOOKUP-GKGTHEMES.txt', 'r')
for line in f.readlines():
    input = line.strip().split('\t')[0]
    table.append(input)
    
# theme list를 클러스터별로 담을 리스트 만들기

partition_set_gkg_list = []
for i in range(len(set(partition5.values()))):
    partition_set_gkg_list.append([])
    
#partition set을 theme list에 맞게 변환 (int 에서 str)
for i in range(len(partition_set)):
    for k in partition_set[i]:
        partition_set_gkg_list[i].append(table[int(k)])
        
print('Clustering properties: ', G5, '\n')

for i in range(len(partition_set_gkg_list)):
    print(f"{i} Cluster's number of nodes: ", len(partition_set_gkg_list[i]))

Clustering properties:  Graph with 3131 nodes and 319127 edges 

0 Cluster's number of nodes:  1588
1 Cluster's number of nodes:  708
2 Cluster's number of nodes:  371
3 Cluster's number of nodes:  464


In [227]:
for i in partition_set_gkg_list[0]:
    print(i)

TAX_FNCACT
MEDIA_MSM
EDUCATION
MANMADE_DISASTER_IMPLIED
GENERAL_HEALTH
MEDICAL
TAX_DISEASE
AFFECT
TAX_FNCACT_CHILDREN
TAX_FNCACT_DIRECTOR
MEDIA_SOCIAL
SCIENCE
NATURAL_DISASTER
TAX_FNCACT_OFFICIALS
TAX_FNCACT_WOMEN
TAX_FNCACT_CHILD
DELAY
MARITIME
TAX_FNCACT_EXECUTIVE
TAX_FNCACT_STUDENTS
AGRICULTURE
TAX_ETHNICITY_AMERICAN
TAX_FNCACT_EMPLOYEES
ECON_STOCKMARKET
TAX_FNCACT_MANAGER
TAX_FNCACT_WORKERS
PUBLIC_TRANSPORT
TAX_FOODSTAPLES
TAX_FNCACT_AUTHOR
ENV_OIL
TAX_FNCACT_REPORTER
TOURISM
TAX_FNCACT_SPOKESMAN
MARITIME_INCIDENT
TAX_ETHNICITY_CHINESE
TAX_FNCACT_KING
TAX_FNCACT_STUDENT
TAX_ETHNICITY_ENGLISH
TAX_FNCACT_DOCTOR
TAX_FNCACT_PROFESSOR
TAX_FNCACT_TEACHER
ECON_ENTREPRENEURSHIP
ECON_HOUSING_PRICES
TAX_FNCACT_CEO
TAX_FNCACT_ACTOR
TAX_FNCACT_GUARD
TAX_FNCACT_JOURNALIST
TAX_FNCACT_WRITER
TAX_FNCACT_EDITOR
TAX_FNCACT_BABY
TAX_FNCACT_DOCTORS
POVERTY
TAX_FNCACT_BOSS
TAX_DISEASE_CANCER
MANMADE_DISASTER
TAX_FNCACT_SPECIALIST
TAX_FNCACT_GUIDE
TAX_FNCACT_FOUNDER
TAX_FNCACT_SPOKESPERSON
TAX_FNCACT_AN

## ebola(after)

In [189]:
G6 = nx.Graph()
file = open('./ebola_after_graph.txt', 'r')
while True:
    line = file.readline()
    if not line:
        break
    line = line.split(',')
    G6.add_edge(str(line[0]), str(line[1]), weight = float(line[2]))
file.close()

# compute the best partition
partition6 = community_louvain.best_partition(G6)



Clustering properties:  Graph with 2679 nodes and 257809 edges 

0 Cluster's number of nodes:  547
1 Cluster's number of nodes:  613
2 Cluster's number of nodes:  1047
3 Cluster's number of nodes:  472


In [228]:
#partition set을 cluster 갯수대로 만들어놓기
partition_set = []
for i in range(len(set(partition6.values()))):
    partition_set.append([])

for k, v in partition6.items():
    partition_set[v].append(k)
    
#theme list 만들기
table = []
f = open('./LOOKUP-GKGTHEMES.txt', 'r')
for line in f.readlines():
    input = line.strip().split('\t')[0]
    table.append(input)
    
# theme list를 클러스터별로 담을 리스트 만들기

partition_set_gkg_list = []
for i in range(len(set(partition6.values()))):
    partition_set_gkg_list.append([])
    
#partition set을 theme list에 맞게 변환 (int 에서 str)
for i in range(len(partition_set)):
    for k in partition_set[i]:
        partition_set_gkg_list[i].append(table[int(k)])
        
print('Clustering properties: ', G6, '\n')

for i in range(len(partition_set_gkg_list)):
    print(f"{i} Cluster's number of nodes: ", len(partition_set_gkg_list[i]))

Clustering properties:  Graph with 2679 nodes and 257809 edges 

0 Cluster's number of nodes:  547
1 Cluster's number of nodes:  613
2 Cluster's number of nodes:  1047
3 Cluster's number of nodes:  472


In [233]:
for i in partition_set_gkg_list[2]:
    print(i)

TAX_FNCACT
MEDIA_MSM
EDUCATION
GENERAL_HEALTH
MEDICAL
TAX_DISEASE
TAX_FNCACT_DIRECTOR
MEDIA_SOCIAL
SCIENCE
NATURAL_DISASTER
TAX_FNCACT_CHIEF
TAX_FNCACT_EXECUTIVE
TAX_FNCACT_STUDENTS
AGRICULTURE
TAX_FNCACT_EMPLOYEES
ECON_STOCKMARKET
TAX_FNCACT_MANAGER
TAX_FNCACT_OFFICER
TAX_FOODSTAPLES
TAX_FNCACT_AUTHOR
ENV_OIL
URBAN
TAX_FNCACT_REPORTER
TOURISM
TAX_MILITARY_TITLE_OFFICER
TAX_ETHNICITY_GERMAN
TRAFFIC
APPOINTMENT
TAX_ETHNICITY_CHINESE
TAX_FNCACT_STUDENT
TAX_FNCACT_DOCTOR
TAX_FNCACT_PROFESSOR
RURAL
TAX_FNCACT_TEACHER
ECON_ENTREPRENEURSHIP
ECON_DEBT
ECON_HOUSING_PRICES
TAX_FNCACT_ASSISTANT
TAX_FNCACT_CEO
TAX_FNCACT_ACTOR
TAX_FNCACT_ARTIST
RETIREMENT
DISABILITY
TAX_FNCACT_WRITER
TAX_FNCACT_EDITOR
TAX_FNCACT_DOCTORS
TAX_FNCACT_BOSS
TAX_DISEASE_CANCER
TAX_FNCACT_VICE_PRESIDENT
TAX_FNCACT_SPECIALIST
TAX_FNCACT_GUIDE
TAX_FNCACT_FOUNDER
TAX_FNCACT_ANALYSTS
TAX_FNCACT_ANALYST
TAX_FNCACT_EMPLOYEE
TAX_FNCACT_AGENT
TAX_FNCACT_ACTRESS
TAX_FNCACT_DRIVERS
WATER_SECURITY
TAX_FNCACT_TOURISTS
RECRUITMENT
T